# Frist create a directory of your name at REPO-ROOT/your-faces/

Write your name in `your_name`

In [3]:
import shutil
import os
HERE = os.getcwd()

your_name = "Tae"

shutil.rmtree(os.path.join(HERE, f'../your-faces/{your_name}/'), ignore_errors=True)
os.makedirs(os.path.join(HERE, f'../your-faces/{your_name}/'))

## Load the modules and define functions

In [4]:
from cltl_face_all.arcface import ArcFace
from cltl_face_all.face_alignment import FaceDetection
from contextlib import contextmanager
import cv2
import numpy as np

@contextmanager
def video_capture(*args, **kwargs):
    cap = cv2.VideoCapture(*args, **kwargs)
    try:
        yield cap
    finally:
        cap.release()


def yield_images():
    # capture video
    with video_capture(0) as cap:
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        while True:
            # get video frame
            ret, img = cap.read()

            if not ret:
                raise RuntimeError("Failed to capture image")

            img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            yield img_RGB

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)


## Instantiate the arcface, and face-detection classes

In [5]:
af = ArcFace(device='cpu')
fd = FaceDetection(device='cpu', face_detector='blazeface')

[*] load ckpt from /home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/cltl_face_all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt


## move around your face. This process ends when 100 faces of yours is well taken.

In [6]:
count = 0
num_to_take = 100
for idx, img in enumerate(yield_images()):
    bboxes = fd.detect_faces(img[np.newaxis, ...])
    landmarks = fd.detect_landmarks(img[np.newaxis, ...], bboxes)
    faces = fd.crop_and_align(img[np.newaxis, ...], bboxes, landmarks)

    # There is only one image per batch. fd returns a list
    bbox = bboxes[0]
    landmark = landmarks[0]
    face = faces[0]

    if len(bbox) == 0:
        continue
    # Only take the first face
    bbox = bbox[0]
    landmark = landmark[0]
    face = face[0]

    # add the dummy axis
    bbox = bbox[np.newaxis, ...]
    landmark = landmark[np.newaxis, ...]
    face = face[np.newaxis, ...]

    face_threshold = 0.85
    
    if len(bbox) > 0 and bbox[0, -1] > face_threshold:
        count +=1

        print(f"{count} / {num_to_take}")
        # ag and af return a np.ndarray
        embeddings = af.predict(face)

        with open(os.path.join(HERE, f'../your-faces/{your_name}/{idx}.npy'), 'wb') as stream:
            np.save(stream, embeddings)

        # print(len(bbox), len(landmark), len(face), len(age), len(gender), len(embeddings))

        for bb, lm in zip(bbox, landmark):
            x1, y1, x2, y2, prob = bb
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

            label = f"{str(round(prob*100, 1))} % face"
            draw_label(img, (x1, y2), label, font_scale=0.5, thickness=1)

            for lm in landmark:
                for xy in lm:
                    cv2.circle(img, (int(xy[0]), int(xy[1])), 1, (0,255,0), -1)

    img = cv2.resize(img, (img.shape[1]*2, img.shape[0]*2))
    cv2.imshow("result", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    key = cv2.waitKey(1)

    for idx, fc in enumerate(face):
        fc = cv2.resize(fc, (fc.shape[1]*2, fc.shape[0]*2))
        cv2.imshow(f"cropped and aligned {idx}", cv2.cvtColor(fc, cv2.COLOR_BGR2RGB))

    if key == 27 or count == num_to_take:  # ESC
        break

cv2.destroyAllWindows()

/home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/cltl_face_all/face_alignment/matlab_cp2tform.py:84: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)


1 / 100
2 / 100
3 / 100
4 / 100
5 / 100
6 / 100
7 / 100
8 / 100
9 / 100
10 / 100
11 / 100
12 / 100
13 / 100
14 / 100
15 / 100
16 / 100
17 / 100
18 / 100
19 / 100
20 / 100
21 / 100
22 / 100
23 / 100
24 / 100
25 / 100
26 / 100
27 / 100
28 / 100
29 / 100
30 / 100
31 / 100
32 / 100
33 / 100
34 / 100
35 / 100
36 / 100
37 / 100
38 / 100
39 / 100
40 / 100
41 / 100
42 / 100
43 / 100
44 / 100
45 / 100
46 / 100
47 / 100
48 / 100
49 / 100
50 / 100
51 / 100
52 / 100
53 / 100
54 / 100
55 / 100
56 / 100
57 / 100
58 / 100
59 / 100
60 / 100
61 / 100
62 / 100
63 / 100
64 / 100
65 / 100
66 / 100
67 / 100
68 / 100
69 / 100
70 / 100
71 / 100
72 / 100
73 / 100
74 / 100
75 / 100
76 / 100
77 / 100
78 / 100
79 / 100
80 / 100
81 / 100
82 / 100
83 / 100
84 / 100
85 / 100
86 / 100
87 / 100
88 / 100
89 / 100
90 / 100
91 / 100
92 / 100
93 / 100
94 / 100
95 / 100
96 / 100
97 / 100
98 / 100
99 / 100
100 / 100


## Load the faces and get the best 50 of them

In [7]:
from glob import glob
from cltl_face_all.arcface import calc_angle_distance

embs_ = glob(os.path.join(HERE, f'../your-faces/{your_name}/*npy'))
embs = []

for emb in embs_:
    with open(emb, 'rb') as stream:
        embs.append(np.load(stream))

embs = np.concatenate(embs)

dists = calc_angle_distance(embs, embs)

indexes = dists.mean(axis=1).argsort()[:len(dists.mean(axis=1)) // 2]

embs = embs[indexes]

emb_mean = embs.mean(axis=0)
emb_final = emb_mean / np.linalg.norm(emb_mean)

with open(os.path.join(HERE, f'../your-faces/{your_name}.npy'), 'wb') as stream:
    np.save(stream, emb_final)

shutil.rmtree(os.path.join(HERE, f'../your-faces/{your_name}/'), ignore_errors=True)